# Tree Methods Code Along

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('College').getOrCreate()
colleges = spark.read.csv('College.csv', header=True, inferSchema=True)

In [2]:
colleges.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [3]:
from pyspark.ml.feature import VectorAssembler

In [4]:
colleges.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [5]:
assembler = VectorAssembler(inputCols=['Apps',
                                       'Accept',
                                       'Enroll',
                                       'Top10perc',
                                       'Top25perc',
                                       'F_Undergrad',
                                       'P_Undergrad',
                                       'Outstate',
                                       'Room_Board',
                                       'Books',
                                       'Personal',
                                       'PhD',
                                       'Terminal',
                                       'S_F_Ratio',
                                       'perc_alumni',
                                       'Expend',
                                       'Grad_Rate'], 
                                outputCol='features')


In [6]:
output = assembler.transform(colleges)

In [7]:
from pyspark.ml.feature import StringIndexer

In [8]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [9]:
output_fixed = indexer.fit(output).transform(output)

In [10]:
colleges_cleaned = output_fixed.select('features', 'PrivateIndex')

In [12]:
colleges_train, colleges_test = colleges_cleaned.randomSplit([0.7, 0.3])

In [13]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [15]:
from pyspark.ml import Pipeline

In [16]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [17]:
dtc_model = dtc.fit(colleges_train)
rfc_model = rfc.fit(colleges_train)
gbt_model = gbt.fit(colleges_train)

In [18]:
dtc_preds = dtc_model.transform(colleges_test)
rfc_preds = rfc_model.transform(colleges_test)
gbt_preds = gbt_model.transform(colleges_test)

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [21]:
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.9197155574856151


In [23]:
print('RFC')
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9816523721637174


In [25]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [26]:
print('GBT')
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8683639127130606


In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [29]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [30]:
rfc_acc

0.9528301886792453